In [1]:
%run ./MovieLens.ipynb

In [2]:
# 设置超参数
d = 20
lambda_ = 0.001
lr = 0.01
epochs = 20

user_num = 943
item_num = 1682
n = 90570

In [3]:
# 初始化参数
def init():
    mu = r_
    bu = np.zeros(user_num + 1, float)
    for u in range(1, user_num + 1):
        bu[u] = (0 if y_ui[u].sum() == 0 else (y_ui[u] * (ratings[u] - mu)).sum() / y_ui[u].sum())
    bi = np.zeros(item_num + 1, float)
    for i in range(1, item_num + 1):
        bi[i] = (0 if y_ui[:,i].sum() == 0 else (y_ui[:,i] * (ratings[:,i] - mu)).sum() / y_ui[:,i].sum())
    U = np.random.rand(user_num + 1, d)
    V = np.random.rand(item_num + 1, d)
    M = np.random.rand(item_num + 1, 6, d)
    U = (U - 0.5) * 0.01
    V = (V - 0.5) * 0.01
    M = (M - 0.5) * 0.01
    return mu, bu, bi, U, V, M

# 求用户u在M上的偏好
def U_MPC(u, M):
    m = np.zeros(d, float)
    for r in R_u(u):
        items = I_u_r(u, r)
        m += np.sum(M[items, r, :], axis=0) / np.sqrt(len(items))
    return m

# 预测函数
def predict_rule(u, i, mu, bu, bi, U, V, M):
    return U[u] @ V[i].T + U_MPC(u, M) @ V[i].T + bu[u] + bi[i]+ mu

In [ ]:
# 训练
mu, bu, bi, U, V, M = init()
train_data = get_train_data()

for epoch in range(epochs):
    print(epoch)
    # 打乱训练集
    indices = list(range(p))
    random.shuffle(indices)
    train_data = train_data.iloc[indices,:]
    for t in range(p):
        # 随机取数据
        u, i, rating = train_data.iloc[t,:]['userId'], train_data.iloc[t,:]['itemId'], train_data.iloc[t,:]['rating']
        # 计算梯度
        e = ratings[u][i] - predict_rule(u, i, mu, bu, bi, U, V, M)
        delta_mu = -e
        delta_bu = -e + lambda_ * bu[u]
        delta_bi = -e + lambda_ * bi[i]
        delta_Uu = (-e) * V[i] + lambda_ * U[u]
        delta_Vi = (-e) * (U[u] + U_MPC(u, M)) + lambda_ * V[i]
        delta_M = np.zeros_like(M)
        for r in R_u(u):
            items = I_u_r(u, r)
            for item in items:
                delta_M[item, r, :] = -e * V[i] / np.sqrt(len(items)) + lambda_ * M[item, r, :]
        # 更新参数
        mu -= lr * delta_mu
        bu[u] -= lr * delta_bu
        bi[i] -= lr * delta_bi
        U[u] -= lr * delta_Uu
        V[i] -= lr * delta_Vi
        for r in R_u(u):
            items = I_u_r(u, r)
            M[items, r, :] -= lr * delta_M[items, r, :]
    # 学习率下降
    lr *= 0.9

In [5]:
# 预测规则
def MF_MPC(u, j):
    return predict_rule(u, j, mu, bu, bi, U, V, M)

In [6]:
# 输出结果
predict(MF_MPC)

RMSE: 0.9185, MAE: 0.7205
